In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 waste_cnt임)

In [3]:
df = pd.read_csv("files\\data_preprocessing\\long_term_frgn_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'long_term_frgn']  
df

,base_date,emd_cd,long_term_frgn
base_date,,,
2018-01-31,2018-01-31,50110250,427986.3363
2018-02-28,2018-02-28,50110250,427240.8119
2018-03-31,2018-03-31,50110250,513638.2631
2018-04-30,2018-04-30,50110250,474601.3510
2018-05-31,2018-05-31,50110250,395262.5483
...,...,...,...
2021-02-28,2021-02-28,50130620,109325.4866
2021-03-31,2021-03-31,50130620,118258.8891
2021-04-30,2021-04-30,50130620,129227.7480


In [4]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)  
data

,emd_cd,long_term_frgn
base_date,,
2018-01-31,50110250,427986.3363
2018-02-28,50110250,427240.8119
2018-03-31,50110250,513638.2631
2018-04-30,50110250,474601.3510
2018-05-31,50110250,395262.5483
...,...,...
2021-02-28,50130620,109325.4866
2021-03-31,50130620,118258.8891
2021-04-30,50130620,129227.7480


In [5]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50110600

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,long_term_frgn
base_date,
2018-01-31,52634.4898
2018-02-28,59151.2122
2018-03-31,65994.1940
2018-04-30,48766.7394
2018-05-31,46068.3523
2018-06-30,67194.4086
2018-07-31,69399.4449
2018-08-31,73117.7761
2018-09-30,60981.7135


In [6]:
# series 형으로 변환
sub_df.to_dict('series')

{'long_term_frgn': base_date
 2018-01-31     52634.4898
 2018-02-28     59151.2122
 2018-03-31     65994.1940
 2018-04-30     48766.7394
 2018-05-31     46068.3523
 2018-06-30     67194.4086
 2018-07-31     69399.4449
 2018-08-31     73117.7761
 2018-09-30     60981.7135
 2018-10-31     76410.6956
 2018-11-30     79629.5311
 2018-12-31     62882.1416
 2019-01-31     81601.7852
 2019-02-28     77170.9812
 2019-03-31     82876.1997
 2019-04-30     95555.0094
 2019-05-31    106183.5452
 2019-06-30    106111.2423
 2019-07-31     97820.6775
 2019-08-31    113886.7094
 2019-09-30     91916.5127
 2019-10-31    103406.5094
 2019-11-30    101058.2775
 2019-12-31    118946.0509
 2020-01-31    123116.0279
 2020-02-29    108502.7504
 2020-03-31    106206.5153
 2020-04-30     84201.7588
 2020-05-31     81375.5468
 2020-06-30     78683.3632
 2020-07-31     86948.8690
 2020-08-31     86734.4213
 2020-09-30     84173.9327
 2020-10-31     88510.3148
 2020-11-30     77344.5370
 2020-12-31     83475.8338

# ARIMA

In [7]:
# ARIMA에 적합한 log로 바꾸는 과정
sub_df_log = np.log1p(sub_df[:])
#sub_df_float.tail()

In [8]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df_log)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -1.541963
p-value : 0.512660
Critical Values :
	1%: -3.639224
	5%: -2.951230
	10%: -2.614447


In [9]:
# 차분
diff_1 = sub_df_log.diff(periods=1).iloc[1:]

# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
result = adfuller(diff_1)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -2.031775
p-value : 0.272817
Critical Values :
	1%: -3.639224
	5%: -2.951230
	10%: -2.614447


In [10]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df_log,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df_log)

 ARIMA(0,1,0)(0,0,0)[1] intercept   : AIC=-42.006, Time=0.13 sec
 ARIMA(0,1,1)(0,0,0)[1] intercept   : AIC=-45.092, Time=0.07 sec
 ARIMA(0,1,2)(0,0,0)[1] intercept   : AIC=-43.600, Time=0.11 sec
 ARIMA(0,1,3)(0,0,0)[1] intercept   : AIC=-42.054, Time=0.18 sec
 ARIMA(0,1,4)(0,0,0)[1] intercept   : AIC=-41.125, Time=0.27 sec
 ARIMA(0,1,5)(0,0,0)[1] intercept   : AIC=-41.204, Time=0.43 sec
 ARIMA(1,1,0)(0,0,0)[1] intercept   : AIC=-43.101, Time=0.06 sec
 ARIMA(1,1,1)(0,0,0)[1] intercept   : AIC=-43.422, Time=0.12 sec
 ARIMA(1,1,2)(0,0,0)[1] intercept   : AIC=inf, Time=0.32 sec
 ARIMA(1,1,3)(0,0,0)[1] intercept   : AIC=-42.878, Time=0.36 sec
 ARIMA(1,1,4)(0,0,0)[1] intercept   : AIC=-40.878, Time=0.49 sec
 ARIMA(2,1,0)(0,0,0)[1] intercept   : AIC=-43.698, Time=0.11 sec
 ARIMA(2,1,1)(0,0,0)[1] intercept   : AIC=-41.916, Time=0.17 sec
 ARIMA(2,1,2)(0,0,0)[1] intercept   : AIC=-42.873, Time=0.43 sec
 ARIMA(2,1,3)(0,0,0)[1] intercept   : AIC=-41.323, Time=0.41 sec
 ARIMA(3,1,0)(0,0,0)[1] inter

ARIMA(order=(0, 1, 1), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [11]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df_log.long_term_frgn.values, order=(0,1,1))    

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

                             ARIMA Model Results                              
Dep. Variable:                    D.y   No. Observations:                   41
Model:                 ARIMA(0, 1, 1)   Log Likelihood                  25.546
Method:                       css-mle   S.D. of innovations              0.129
Date:                Sat, 11 Sep 2021   AIC                            -45.092
Time:                        04:11:59   BIC                            -39.952
Sample:                             1   HQIC                           -43.220
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0098      0.012      0.808      0.419      -0.014       0.033
ma.L1.D.y     -0.4135      0.153     -2.708      0.007      -0.713      -0.114
                                    Roots           

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


# 예측하기

In [12]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([11.32260733, 11.33236104]), array([0.12947065, 0.15009886]), array([[11.06884951, 11.57636515],
       [11.03817269, 11.6265494 ]]))


In [13]:
print( np.exp(11.32260733) - 1 )   
print( np.exp(11.33236104) - 1 )  

82668.60911676768
83478.88970703677
